In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install wandb
!pip install datasets
!pip install evaluate

from tqdm import tqdm, trange
import multiprocessing
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import torch
import os

import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import accelerate
import transformers


transformers.__version__, accelerate.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Attem

('4.37.2', '0.27.2')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir("/content/drive/My Drive/LSEG_Dataset")

file_path = "/content/drive/My Drive/LSEG_Dataset/cwe476_Data_op.csv"
df = pd.read_csv(file_path)


Mounted at /content/drive


In [ ]:
df.head()


,code,CWE-476,Label
0,Perl_mro_get_linear_isa(pTHX_ HV *pTHX_Pointe...,True,1
1,handlePRERET(FlowInstruction *FlowInstruction...,True,1
2,GetInetAddr(char *charPointerMarker)\n\n{ str...,True,1
3,do_redirect(action_list_t *action_list_tPoint...,True,1
4,gf_af_new(GF_Compositor *GF_CompositorPointer...,True,1


In [1]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
dataset = {}
dataset['text'] = list(df_train['code'])
dataset['label'] = list(df_train['Label'])
ds = Dataset.from_dict(dataset)


NameError: name 'train_test_split' is not defined

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/7475 [00:00<?, ? examples/s]

Map:   0%|          | 0/831 [00:00<?, ? examples/s]

In [ ]:
 #Assuming you have imported the necessary libraries and assigned your DatasetDict to a variable named 'dataset_dict'

for split_name, split_dataset in tokenized_dataset.items():
    print(f"Split: {split_name}")
    print(f"Number of rows: {split_dataset.num_rows}")
    print(f"Features: {split_dataset.features}")

    # Iterate through each entry in the dataset
    for i in range(10):
        entry = split_dataset[i]
        print(f"Entry {i}: {entry}")
    print("\n")

Split: train
Number of rows: 7475
Features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
Entry 0: {'text': ' symsize(unsigned unsignedMarker, const struct huftrees *structPointerMarker)\n{\n    unsigned basesym = unsignedMarker &~ SYMPFX_MASK;\n\n    switch (unsignedMarker & SYMPFX_MASK) {\n      case SYMPFX_LITLEN:\n return structPointerMarker->len_litlen[basesym];\n      case SYMPFX_DIST:\n return structPointerMarker->len_dist[basesym];\n      case SYMPFX_CODELEN:\n return structPointerMarker->len_codelen[basesym];\n      default :\n return basesym >> SYM_EXTRABITS_SHIFT;\n    }\n}\n\n', 'label': 0, 'input_ids': [0, 13550, 4339, 2072, 1640, 49418, 39023, 10006, 254, 6, 10759, 29916, 1368, 257, 2543, 13448, 1009, 25384, 26170, 8007, 10006, 254, 43, 50118, 45152, 50118, 1437, 1437, 143

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base-mlm")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.304000,0.206233,0.933815
2,0.246300,0.247741,0.943442


TrainOutput(global_step=1870, training_loss=0.27289765301872704, metrics={'train_runtime': 1896.6906, 'train_samples_per_second': 7.882, 'train_steps_per_second': 0.986, 'total_flos': 3900308951333760.0, 'train_loss': 0.27289765301872704, 'epoch': 2.0})

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

preds = []
for i in df_test['code'].values:
    with torch.no_grad():
        inputs = tokenizer(i, return_tensors="pt",  truncation=True).to(device)
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        preds.append(predicted_class_id)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_test['Label'],preds))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1883
           1       0.88      0.07      0.13       194

    accuracy                           0.91      2077
   macro avg       0.89      0.54      0.54      2077
weighted avg       0.91      0.91      0.88      2077



In [ ]:
count = 0
for x in preds:
  if x==1:
    count+=1


print(count)

16
